**Step0 : Import baisic**

In [ ]:
#데이터 관리
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
#모델
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error,r2_score
import lightgbm as lgb

**Step1 : input data and check**

In [ ]:
#데이터불러오기
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
train.head()

In [ ]:
#간단히 확인
train.info()

**Step2 : ready for model**

In [ ]:
test1=test.drop('Id',axis=1)   #id값은 따로 특징이 아니기떄문에 모델에서 제거

In [ ]:
cols=['Id','Pawpularity']        #좀더 쉽게  제거하기위해 만든 변수
x_train=train.drop(cols,axis=1)  #id 와 pawpularity를 제거한 특정값들만 남긴다
y_train=train['Pawpularity']     #찾고자하는 값

In [ ]:
# train_test_split를 사용하여 train 데이터와 test 데이터를 나눈다
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)  

In [ ]:
#xgb모델만들기
xgb=XGBRegressor()   #우리가 예측할 범위는 (0~무한대)인 특정 값이므로 regressor가 어울리니 regressor를 생성해준다.
xgb.fit(x_train,y_train)   #fit을 이용하여 모델에 값을 넣어준다
y_pred=xgb.predict(x_test)  #예측값을 찾는다.

In [ ]:
def rmse(y, y_pred):    #rmse = 루트 평균 제곱 오차 (RMSE를 사용하면 오류 지표를 실제 값과 유사한 단위로 다시 변환하여 해석을 쉽게 합니다)
    return np.sqrt(mean_squared_error(y, y_pred))
print(rmse(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

In [ ]:
#lightGBM 모델 만들기
#회귀를 위한 LGBMRegressor(max_depth=나무깊이,learning_rate=훈련양(0.005~0.1),n_estimators=200,reg_alpha=1,reg_lambda=0.1)
lgbr=lgb.LGBMRegressor(max_depth=10,learning_rate=0.005,n_estimators=200,reg_alpha=1,reg_lambda=0.1)
lgbr.fit(x_train,y_train)   #fit을 이용하여 모델에 값을 넣어준다
y_pred=lgbr.predict(x_test) #예측값을 찾는다

In [ ]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))
print(rmse(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

In [ ]:
Pred=lgbr.predict(test1)  #예측값을 찾는다

In [ ]:
#최종결과물생성
sub=pd.DataFrame()
sub['Id']=test['Id']
sub['Pawpularity']=Pred
sub.to_csv('submission.csv',index=False)
print(sub.head())